## Running multiple SigMApy analysis

In [6]:
import os
import pandas as pd
import time
import warnings
import sigmapy as sgpy
import numpy as np
import glob

# Project paths
project_folder = "/home/ferperez/SigMA_tests/"
input_path = "/home/ferperez/SigMA_tests/Input_datasets/"
output_path = "/home/ferperez/SigMA_tests/Output_SigMA_Py/"

# Listing subdirectories in Input_datasets
all_dirs = [os.path.join(dp, d) for dp, dn, filenames in os.walk(input_path) for d in dn]
len_dir_depth = [len(path.split("/")) for path in all_dirs]
max_depth = max(len_dir_depth)
input_dirs = [d for d, depth in zip(all_dirs, len_dir_depth) if depth == max_depth]

print("Input directories:")
print(input_dirs)

#Creating dictionary for matching input data with SigMA datasets
samples_dict = pd.DataFrame({
    "tumor_folder": ["BRCA", "COADREAD", "PAAD"],
    "tumor_abreviation": ["breast", "crc", "panc_ad"]
})

sequencing_dict = pd.DataFrame({
    "seq_folder": ["MSK_IMPACTv1", "mc3"],
    "seq_abreviation": ["msk", "tcga_mc3"]
})

Input directories:
['/home/ferperez/SigMA_tests/Input_datasets/BRCA/mc3', '/home/ferperez/SigMA_tests/Input_datasets/BRCA/Foundation_One', '/home/ferperez/SigMA_tests/Input_datasets/BRCA/MSK_IMPACTv1', '/home/ferperez/SigMA_tests/Input_datasets/BRCA/MSK_IMPACTv2', '/home/ferperez/SigMA_tests/Input_datasets/PAAD/mc3', '/home/ferperez/SigMA_tests/Input_datasets/PAAD/SeqCap_capture_targets', '/home/ferperez/SigMA_tests/Input_datasets/COADREAD/Foundation_One', '/home/ferperez/SigMA_tests/Input_datasets/COADREAD/MSK_IMPACTv1', '/home/ferperez/SigMA_tests/Input_datasets/COADREAD/MSK_IMPACTv2']


In [2]:
# Running SigMA directory by directory
for mydir in input_dirs:
    parts = mydir.split("/")
    seq_type = parts[-1]
    sample_type = parts[-2]

    seq_type_for_SigMA = sequencing_dict.loc[sequencing_dict["seq_folder"] == seq_type, "seq_abreviation"]
    sample_type_for_SigMA = samples_dict.loc[samples_dict["tumor_folder"] == sample_type, "tumor_abreviation"]

    # In case the input folders are not supported by SigMA
    if seq_type_for_SigMA.empty or sample_type_for_SigMA.empty:
        #print("Ignoring the next dir because of lack of matching in list_tumor_types()")
        #print(mydir)
        continue
    print(f"Starting to analyse: {mydir}")

    # Defining the output folder for storing results
    output_dir_iter = os.path.join(output_path, sample_type, seq_type)
    os.makedirs(output_dir_iter, exist_ok=True)

    seq_type_for_SigMA = seq_type_for_SigMA.values[0]
    sample_type_for_SigMA = sample_type_for_SigMA.values[0]

    # The datasets in mc3 folder are MAF, otherwise VCF
    if seq_type_for_SigMA == "tcga_mc3":
        data_type = "maf"
        mydir = os.path.join(mydir, f"{sample_type_for_SigMA}.maf")
    else:
        data_type = "vcf"

    # Generating count matrix
    #Running matrix class from sgpy
    matrix_instance = sgpy.Matrix(mydir,
                                    ref_genome="hg19")
    mut_counts_file = os.path.join(output_dir_iter, "Mut_counts.csv")

    matrix_instance.df.to_csv(mut_counts_file, sep=",", index=True)
    print(f"96-dimensional matrix is saved in {mut_counts_file}")
    
    print("Running SigMA")

    start_time = time.perf_counter()

    sigMA_output = sgpy.run(
                    input=mydir,
                    tumor_type=sample_type_for_SigMA,
                    ref_genome="hg19",
                    mxw=12,
                    soi="SBS3",
                    mva=True,
                    seq_type=seq_type_for_SigMA,
                    catalog="cosmic_v2_inhouse",
                    msi=False)
    
    sigMA_output.df.to_csv(os.path.join(output_dir_iter, "SigMA_output.csv"), sep=",", index=True)
    end_time = time.perf_counter()
    print(f"SigMA analysis completed in {end_time - start_time:.2f} seconds.")


Starting to analyse: /home/ferperez/SigMA_tests/Input_datasets/BRCA/mc3
96-dimensional matrix is saved in /home/ferperez/SigMA_tests/Output_SigMA_Py/BRCA/mc3/Mut_counts.csv
Running SigMA


Warning message:
package ‘gbm’ was built under R version 4.3.3 


SigMA analysis completed in 107.47 seconds.
Starting to analyse: /home/ferperez/SigMA_tests/Input_datasets/BRCA/MSK_IMPACTv1
96-dimensional matrix is saved in /home/ferperez/SigMA_tests/Output_SigMA_Py/BRCA/MSK_IMPACTv1/Mut_counts.csv
Running SigMA


Warning message:
package ‘gbm’ was built under R version 4.3.3 


SigMA analysis completed in 22.09 seconds.
Starting to analyse: /home/ferperez/SigMA_tests/Input_datasets/PAAD/mc3


/home/ferperez/SigMApy/sigmapy/matrix.py:606: DtypeWarning: Columns (4,38,57,63,74,78) have mixed types. Specify dtype option on import or set low_memory=False.
  maf = pd.read_csv(files[0], comment='#', sep='\t', header=0, encoding='utf-8')


96-dimensional matrix is saved in /home/ferperez/SigMA_tests/Output_SigMA_Py/PAAD/mc3/Mut_counts.csv
Running SigMA


/home/ferperez/SigMApy/sigmapy/matrix.py:606: DtypeWarning: Columns (4,38,57,63,74,78) have mixed types. Specify dtype option on import or set low_memory=False.
  maf = pd.read_csv(files[0], comment='#', sep='\t', header=0, encoding='utf-8')
Warning message:
package ‘gbm’ was built under R version 4.3.3 


SigMA analysis completed in 18.80 seconds.
Starting to analyse: /home/ferperez/SigMA_tests/Input_datasets/COADREAD/MSK_IMPACTv1
96-dimensional matrix is saved in /home/ferperez/SigMA_tests/Output_SigMA_Py/COADREAD/MSK_IMPACTv1/Mut_counts.csv
Running SigMA
SigMA analysis completed in 13.57 seconds.


Error in eval(predvars, data, env) : 
  object 'Signature_3_c1_ml_ml' not found
Calls: source ... predict.gbm -> model.frame -> model.frame.default -> eval -> eval
In addition: Warning message:
package ‘gbm’ was built under R version 4.3.3 
Execution halted
/home/ferperez/SigMApy/sigmapy/tools/mva_predictor.py:58: UserWarning: MVA prediction failed: Command '['Rscript', '-e', '\n            args <- c("/tmp/tmpw3kkwl51.csv", "/tmp/tmpqmq5m1gz.csv","Signature_3", "msk", "crc", "0", "0", "/home/ferperez/SigMApy/sigmapy/var/sysdata.rda");\n            \n            source("/home/ferperez/SigMApy/sigmapy/r/predict_mva.R");\n            ']' returned non-zero exit status 1.
  warnings.warn(f"MVA prediction failed: {e}")


### Comparing now the results from SigMApy vs SigMAR

Auxiliar functions:

In [3]:
#Function to compare the differences between two SNV matrixs when they are not equal
def compare_dataframes(df1, df2):
    # Ensure same index and columns order
    df1 = df1.sort_index().sort_index(axis=1)
    df2 = df2.sort_index().sort_index(axis=1)

    # Check shape and columns
    if not (df1.shape == df2.shape and (df1.columns == df2.columns).all() and (df1.index == df2.index).all()):
        print("DataFrames do not have the same shape, columns, or index.")
        return
    
    # Find differences
    diff = df1 != df2
    n_diff = diff.values.sum()
    print(f"Number of discordant values: {n_diff}")
    if n_diff > 0:
        rows, cols = diff.to_numpy().nonzero()
        for r, c in zip(rows, cols):
            print(f"Row: {df1.index[r]}, Column: {df1.columns[c]}, SigmaPY: {df1.iat[r, c]}, SigmaR: {df2.iat[r, c]}")
    else:
        print("All values are concordant.")



# Comparing the outputs from SigMA Python and SigMA R
def mean_diff_exps(df1, df2):
    # Calculate mean absolute difference for 'exps_all' columns
    diffs = []
    for py, r in zip(df1['exps_all'], df2['exps_all']):
        py_vals = np.array([float(x) for x in str(py).split('_')])
        r_vals = np.array([float(x) for x in str(r).split('_')])
        # Ensure same length
        if len(py_vals) == len(r_vals):
            diffs.extend(np.abs(py_vals - r_vals))
        else:
            print("Row length mismatch, skipping.")
    mean_diff = np.mean(diffs)
    return(mean_diff)


##### Comparing if the SNV count matrix are the same

In [4]:
#Reading the output files from SigMA R version and Python verision

R_output_path = "/home/ferperez/SigMA_tests/Output_SigMA_Rversion/"


for mydir in input_dirs:
    parts = mydir.split("/")
    seq_type = parts[-1]
    sample_type = parts[-2]

    seq_type_for_SigMA = sequencing_dict.loc[sequencing_dict["seq_folder"] == seq_type, "seq_abreviation"]
    sample_type_for_SigMA = samples_dict.loc[samples_dict["tumor_folder"] == sample_type, "tumor_abreviation"]

    # In case the input folders are not supported by SigMA
    if seq_type_for_SigMA.empty or sample_type_for_SigMA.empty:
        continue
    
    print(mydir)

    # Reading and sorting the SNV count matrix generated by SigMA Python version
    output_dir_iter = os.path.join(output_path, sample_type, seq_type)
    SigmaPY_matrix_out = pd.read_csv(os.path.join(output_dir_iter, "Mut_counts.csv"))
    SigmaPY_matrix_out.index = SigmaPY_matrix_out['Unnamed: 0'].apply(os.path.basename)
    SigmaPY_matrix_out = SigmaPY_matrix_out.drop('Unnamed: 0', axis=1)
    SigmaPY_matrix_out = SigmaPY_matrix_out.sort_index().sort_index(axis=1)

    # Reading and sorting the SNV count matrix generated by SigMA R version
    R_output_dir_iter = os.path.join(R_output_path, sample_type, seq_type)
    SigmaR_matrix_out = pd.read_csv(os.path.join(R_output_dir_iter, "Mut_counts.csv"))
    SigmaR_matrix_out.index = SigmaR_matrix_out['tumor'].apply(os.path.basename)
    SigmaR_matrix_out = SigmaR_matrix_out.drop('tumor', axis=1)
    SigmaR_matrix_out = SigmaR_matrix_out.sort_index().sort_index(axis=1)

    Are_equal = SigmaPY_matrix_out.equals(SigmaR_matrix_out)
    
    print(f"Are the 96-dimensional matrices from SigMA_Python and SigMA_R equal? {Are_equal}")

    if (Are_equal == False):
        print("Comparing the discordant values:")
        compare_dataframes(SigmaPY_matrix_out, SigmaR_matrix_out)


/home/ferperez/SigMA_tests/Input_datasets/BRCA/mc3
Are the 96-dimensional matrices from SigMA_Python and SigMA_R equal? True
/home/ferperez/SigMA_tests/Input_datasets/BRCA/MSK_IMPACTv1
Are the 96-dimensional matrices from SigMA_Python and SigMA_R equal? True
/home/ferperez/SigMA_tests/Input_datasets/PAAD/mc3
Are the 96-dimensional matrices from SigMA_Python and SigMA_R equal? False
Comparing the discordant values:
Number of discordant values: 91
Row: TCGA-IB-7651-01A-11D-2154-08, Column: caaa, SigmaPY: 0, SigmaR: 3
Row: TCGA-IB-7651-01A-11D-2154-08, Column: caac, SigmaPY: 11, SigmaR: 71
Row: TCGA-IB-7651-01A-11D-2154-08, Column: caag, SigmaPY: 2, SigmaR: 4
Row: TCGA-IB-7651-01A-11D-2154-08, Column: caat, SigmaPY: 51, SigmaR: 246
Row: TCGA-IB-7651-01A-11D-2154-08, Column: caca, SigmaPY: 12, SigmaR: 65
Row: TCGA-IB-7651-01A-11D-2154-08, Column: cacc, SigmaPY: 26, SigmaR: 161
Row: TCGA-IB-7651-01A-11D-2154-08, Column: cacg, SigmaPY: 3, SigmaR: 32
Row: TCGA-IB-7651-01A-11D-2154-08, Column:

##### Comparing how similar the exposures and Sig3 clasifications are

In [7]:
#Reading the output files from SigMA R version and Python verision

R_output_path = "/home/ferperez/SigMA_tests/Output_SigMA_Rversion/"
comparison_results = pd.DataFrame(columns=["Input_folder", "seq_type",
                                            "sample_type", "N_samples_R",
                                            "N_samples_Python",
                                            "Concordant_sigs", "Disconcordant_sigs",
                                            "Mean_exp_difference"])


for mydir in input_dirs:
    parts = mydir.split("/")
    seq_type = parts[-1]
    sample_type = parts[-2]

    seq_type_for_SigMA = sequencing_dict.loc[sequencing_dict["seq_folder"] == seq_type, "seq_abreviation"]
    sample_type_for_SigMA = samples_dict.loc[samples_dict["tumor_folder"] == sample_type, "tumor_abreviation"]

    # In case the input folders are not supported by SigMA
    if seq_type_for_SigMA.empty or sample_type_for_SigMA.empty:
        continue
    
    print(mydir)

    # Reading and sorting the SNV count matrix generated by SigMA Python version
    output_dir_iter = os.path.join(output_path, sample_type, seq_type)
    SigmaPY_out = pd.read_csv(os.path.join(output_dir_iter, "SigMA_output.csv"))
    SigmaPY_out.index = SigmaPY_out['Unnamed: 0'].apply(os.path.basename)
    SigmaPY_out = SigmaPY_out.drop('Unnamed: 0', axis=1)
    SigmaPY_out = SigmaPY_out.sort_index().sort_index(axis=1)

    # Reading and sorting the SNV count matrix generated by SigMA R version
    R_output_dir_iter = os.path.join(R_output_path, sample_type, seq_type)
    lite_files = glob.glob(os.path.join(R_output_dir_iter, "*_lite.csv"))
    SigmaR_out = pd.read_csv(lite_files[0])
    SigmaR_out.index = SigmaR_out['tumor'].apply(os.path.basename)
    SigmaR_out = SigmaR_out.drop('tumor', axis=1)
    SigmaR_out = SigmaR_out.sort_index().sort_index(axis=1)

    #Countin how many samples are per result,
    #The results does not always have the same samples
    N_samples_R = SigmaR_out.shape[0]
    N_samples_Python = SigmaPY_out.shape[0]
    common_idx = SigmaPY_out.index.intersection(SigmaR_out.index)


    Concordant_samples = (SigmaPY_out.loc[common_idx, 'sigs_all'] == SigmaR_out.loc[common_idx, 'sigs_all']).sum()
    Disconcordant_samples = (SigmaPY_out.loc[common_idx, 'sigs_all'] != SigmaR_out.loc[common_idx, 'sigs_all']).sum()


    mean_difference = mean_diff_exps(SigmaPY_out.loc[common_idx], SigmaR_out.loc[common_idx])


    # Store results in a DataFrame
    comparison_results.loc[len(comparison_results)] = [mydir, seq_type_for_SigMA.values[0],
                                                        sample_type_for_SigMA.values[0],
                                                        N_samples_R, N_samples_Python,
                                                        Concordant_samples,
                                                        Disconcordant_samples,
                                                        mean_difference]
comparison_results

/home/ferperez/SigMA_tests/Input_datasets/BRCA/mc3
/home/ferperez/SigMA_tests/Input_datasets/BRCA/MSK_IMPACTv1
/home/ferperez/SigMA_tests/Input_datasets/PAAD/mc3
/home/ferperez/SigMA_tests/Input_datasets/COADREAD/MSK_IMPACTv1


,Input_folder,seq_type,sample_type,N_samples_R,N_samples_Python,Concordant_sigs,Disconcordant_sigs,Mean_exp_difference
0,/home/ferperez/SigMA_tests/Input_datasets/BRCA...,tcga_mc3,breast,985,982,982,0,5.358043e-14
1,/home/ferperez/SigMA_tests/Input_datasets/BRCA...,msk,breast,127,127,127,0,1.176575e-14
2,/home/ferperez/SigMA_tests/Input_datasets/PAAD...,tcga_mc3,panc_ad,149,149,149,0,2.074130e+01
3,/home/ferperez/SigMA_tests/Input_datasets/COAD...,msk,crc,139,139,139,0,1.251579e-14
